In [18]:
from werkzeug.wrappers import Request, Response
from flask import Flask,redirect, url_for, request,flash
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import os
from werkzeug.utils import secure_filename
import requests
from flask import Flask, render_template, request
import tika,re
from tika import parser


app = Flask(__name__, template_folder='template')
UPLOAD_FOLDER = 'template'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS



@app.route('/')
def upload_form():
    return render_template('upload.html')

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            tika.initVM()
            file_data = parser.from_file(filename)
            article_text = file_data['content']
    
            article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
            article_text = re.sub(r'\s+', ' ', article_text) 
            formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
            formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
            sentence_list = nltk.sent_tokenize(article_text)
            clean_sentences = pd.Series(sentence_list).str.replace("[^a-zA-Z]", " ")
            clean_sentences = [s.lower() for s in clean_sentences]
            stop_words = stopwords.words('english')
            def remove_stopwords(sen):
                sen_new = " ".join([i for i in sen if i not in stop_words])
                return sen_new
            clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

            word_embeddings = {}
            f = open('C:/users/Videet/1/glove.6B.100d.txt', encoding='utf-8')
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                word_embeddings[word] = coefs
            f.close()

            sentence_vectors = []
            for i in clean_sentences:
                if len(i) != 0:
                    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
                else:
                    v = np.zeros((100,))
                sentence_vectors.append(v)
            sim_mat = np.zeros([len(sentence_list), len(sentence_list)])
            for i in range(len(sentence_list)):
                for j in range(len(sentence_list)):
                    if i != j:
                        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
            nx_graph = nx.from_numpy_array(sim_mat)
            scores = nx.pagerank(nx_graph)
            ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_list)), reverse=True)
            for i in range(1):
                return(ranked_sentences[i][1])
            return redirect('/')
        else:
            flash('Allowed file types are txt, pdf, png, jpg, jpeg, gif')
            return redirect(request.url)


    
if __name__ == '__main__':
    
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jul/2019 21:23:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2019 21:24:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2019 21:25:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2019 21:25:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2019 21:25:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2019 21:25:46] "POST / HTTP/1.1" 200 -
